In [1]:
%%spark
spark.sql("CREATE DATABASE IF NOT EXISTS tipspark")
spark.sql("CREATE TABLE IF NOT EXISTS tipspark.date_dim USING DELTA LOCATION 'abfss://commonzone@synapsetiphackathon.dfs.core.windows.net/enricheddata/tiphack/date_dim/AutoLoader/data/'")

In [2]:
%%pyspark
dfdim = spark.sql("SELECT * from tipspark.date_dim")
display(dfdim.limit(10))

In [3]:
%%pyspark
dfsales = spark.sql("SELECT * from tipspark.catalog_sales")
display(dfsales.limit(10))


In [4]:
%%pyspark
df = spark.sql("SELECT * from tipspark.catalog_sales, tipspark.date_dim WHERE tipspark.catalog_sales.cs_sold_date_sk = tipspark.date_dim.d_date_sk")
display(df.limit(10))

In [5]:
%%pyspark

cols = ("d_date_sk","d_date_id","d_month_seq","d_week_seq","d_quarter_seq","d_year","d_dow","d_moy","d_dom","d_qoy","d_fy_year","d_fy_quarter_seq","d_fy_week_seq","d_day_name","d_quarter_name","d_holiday","d_weekend","d_following_holiday","d_first_dom","d_last_dom","d_same_day_ly","d_same_day_lq","d_current_day","d_current_week","d_current_month","d_current_quarter","d_current_year")
          
df2 = df.drop(*cols)
df2.printSchema()
display(df2.limit(10))

In [21]:
%%pyspark

from pyspark.sql.functions import split

# split() function defining parameters
split_cols = split(df2['d_date'], '-')
  
# Now applying split() using withColumn()
dfsplit = df2.withColumn('year', split_cols.getItem(0)) \
    .withColumn('month', split_cols.getItem(1)) \
    .withColumn('day', split_cols.getItem(2))
    
  
# display df
display(dfsplit.limit(10))

In [34]:
from pyspark.sql.functions import to_date, concat, lit, col
from pyspark.sql.types import IntegerType

dffinal1 = dfsplit.withColumn('d_date_new',to_date(dfsplit.d_date, 'yyyy-MM-dd'))
dffinal2 = dffinal1.select("*", concat(col("year"),col("month"),col("day")).alias("date_integer"))
dffinal2 = dffinal2.withColumn("date_integer",dffinal2["date_integer"].cast(IntegerType()))
dffinal2.printSchema()
display(dffinal2.limit(10))

In [28]:
# Parquet
dffinal2.write.format('parquet')\
  .mode('overwrite')\
  .partitionBy ("year", "month", "day")\
  .save('abfss://commonzone@synapsetiphackathon.dfs.core.windows.net/enricheddata/tiphackparquet/catalog_sales')

In [30]:
%%pyspark
dfparq = spark.read.load('abfss://commonzone@synapsetiphackathon.dfs.core.windows.net/enricheddata/tiphackparquet/catalog_sales', format='parquet')
dfparq.printSchema()
display(dfparq.limit(10))

In [38]:
#  Delta
dffinal2.write.format('delta')\
  .mode('overwrite')\
  .option("overwriteSchema", "true")\
  .partitionBy ("year", "month", "day")\
  .save('abfss://commonzone@synapsetiphackathon.dfs.core.windows.net/enricheddata/tiphackdelta/catalog_sales')

In [27]:
%%pyspark

spark.sql("CREATE DATABASE IF NOT EXISTS tipspark")
spark.sql("CREATE TABLE IF NOT EXISTS tipspark.catalog_sales_parquet USING PARQUET LOCATION 'abfss://commonzone@synapsetiphackathon.dfs.core.windows.net/enricheddata/tiphackparquet/catalog_sales/*/*/*/*.snappy.parquet'")
dfdisp = spark.sql("select * from tipspark.catalog_sales_parquet")

display(dfdisp.limit(10))

In [6]:
%%pyspark

spark.sql("CREATE DATABASE IF NOT EXISTS tipspark")
spark.sql("CREATE TABLE IF NOT EXISTS tipspark.catalog_sales_delta USING DELTA LOCATION 'abfss://commonzone@synapsetiphackathon.dfs.core.windows.net/enricheddata/tiphackdelta/catalog_sales/'")
dfdisp = spark.sql("select * from tipspark.catalog_sales_delta")

display(dfdisp.limit(10))

In [8]:
%%sql

select count(*) from tipspark.catalog_sales_delta where year = 2001

In [39]:
%%pyspark
dfdelta = spark.read.load('abfss://commonzone@synapsetiphackathon.dfs.core.windows.net/enricheddata/tiphackdelta/catalog_sales', format='delta')
display(dfdelta.limit(10))
dfdelta.printSchema()